In [ ]:
!pip install \
langchain_community \
langchain_pinecone \
langchain_openai \
unstructured \
langchain-text-splitters

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import glob

In [ ]:
!pip install pypdf

In [ ]:
def read_doc(directory):
    from langchain_community.document_loaders import PyPDFLoader # import PyPDFLoader within the function
    loaders = [PyPDFLoader(os.path.join(directory, fn)) for fn in os.listdir(directory)] # create a list of PyPDFLoaders
    documents = []
    for loader in loaders:
        documents.extend(loader.load()) # load documents from each loader
    return documents

doc=read_doc('/content/sample_data/data')
doc

[Document(metadata={'source': '/content/sample_data/data/Unit-4_DL.pdf', 'page': 0}, page_content='Unit -4 \n \nWord similarity is a fundamental concept in natural language processing, and there are several \napplications where measuring the similarity or relatedness between words is crucial. Some of these \napplications include:  \n1. Information Retrieval : In information retrieval systems, understanding the similarity between \nuser queries and documents is essential for ranking search results. Word similarity measures can \nhelp identify documents that are semantically related to a user\'s query.  \n2. Text Summarization : When generating text summaries, it\'s important to ensure that the \nsummary captures the key information in a document. Word similarity can be used to identify \nimportant words and phrases in the source text and select them for inclusion in the summary.  \n3. Mac hine Translation : In machine translation systems, word similarity can be useful for aligning \nwor

In [ ]:
doc[0]

Document(metadata={'source': '/content/sample_data/data/Unit-4_DL.pdf', 'page': 0}, page_content='Unit -4 \n \nWord similarity is a fundamental concept in natural language processing, and there are several \napplications where measuring the similarity or relatedness between words is crucial. Some of these \napplications include:  \n1. Information Retrieval : In information retrieval systems, understanding the similarity between \nuser queries and documents is essential for ranking search results. Word similarity measures can \nhelp identify documents that are semantically related to a user\'s query.  \n2. Text Summarization : When generating text summaries, it\'s important to ensure that the \nsummary captures the key information in a document. Word similarity can be used to identify \nimportant words and phrases in the source text and select them for inclusion in the summary.  \n3. Mac hine Translation : In machine translation systems, word similarity can be useful for aligning \nword

In [ ]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

index_name = "ragchatbot"

## Split our document into chunk

text_splitter = RecursiveCharacterTextSplitter()
split_docs = text_splitter.split_documents(doc)

In [ ]:
from re import split
split_docs[0]

Document(metadata={'source': '/content/sample_data/data/Unit-4_DL.pdf', 'page': 0}, page_content='Unit -4 \n \nWord similarity is a fundamental concept in natural language processing, and there are several \napplications where measuring the similarity or relatedness between words is crucial. Some of these \napplications include:  \n1. Information Retrieval : In information retrieval systems, understanding the similarity between \nuser queries and documents is essential for ranking search results. Word similarity measures can \nhelp identify documents that are semantically related to a user\'s query.  \n2. Text Summarization : When generating text summaries, it\'s important to ensure that the \nsummary captures the key information in a document. Word similarity can be used to identify \nimportant words and phrases in the source text and select them for inclusion in the summary.  \n3. Mac hine Translation : In machine translation systems, word similarity can be useful for aligning \nword

In [ ]:
vectorstores = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)

In [ ]:
query = "what is neural network?"

similar_docs = vectorstores.similarity_search(query)

In [ ]:
similar_docs

[Document(id='8c2ee895-1705-4084-a931-ba054c5626f0', metadata={'page': 13.0, 'source': '/content/sample_data/data/unit-1_DL.pdf'}, page_content='Typically, dropout is applied to the hidden layers of a neural network, and the dropout rate is a \nhyperparameter that determines the probability of dropping out each neuron (common values are around \n0.2 to 0.5). During inference or when making predictions, dropout is usually turned off, and the entire \nnetwork is used.  \nIn summary, regularization techniques like L1 and L2 regularization add penalties to the loss function \nbased on the magnitude of the weights to prevent overfitting. Dropout, on the other hand, randomly \ndeactivates neurons during training to encourage the n etwork to learn more robust features and reduce \noverfitting. These techniques are essential tools for improving the generalization and performance of \nneural networks on unseen data.  \nConvolutional Neural Networks  \n \nPeople are getting more fascinated with 

In [ ]:
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstores.as_retriever()
)

qa.invoke(query)

<ipython-input-46-8bd867ab0945>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


{'query': 'what is neural network?',
 'result': "A neural network, also known as an artificial neural network (ANN), is a computational model inspired by the way biological neural networks in the human brain process information. It consists of interconnected nodes, or neurons, organized in layers. These layers include an input layer, one or more hidden layers, and an output layer. Each connection between neurons has an associated weight, which is adjusted during the training process to minimize the error in the network's predictions.\n\nHere are some key characteristics and components of neural networks:\n\n1. **Layers**:\n   - **Input Layer**: This is where data is fed into the network. Each node in the input layer represents a feature or input variable.\n   - **Hidden Layers**: These are intermediate layers between the input and output layers. Each neuron in a hidden layer performs a weighted sum of its inputs and applies an activation function to produce an output. The presence of h

In [ ]:
qa.invoke({"query": "what is deep learning?"})

{'query': 'what is deep learning?',
 'result': 'Deep learning is a subset of machine learning that involves the use of artificial neural networks with many layers (hence "deep") to model and understand complex patterns in data. It is particularly effective for tasks where traditional machine learning algorithms struggle, such as image and speech recognition, natural language processing, and more. Here are some key aspects of deep learning:\n\n1. **Neural Networks**: Deep learning models are based on neural networks, which are computational structures inspired by the human brain. These networks consist of layers of interconnected nodes (neurons) that process input data and learn to make predictions or classifications.\n\n2. **Multiple Layers**: Unlike traditional neural networks, deep learning models have many layers (hence "deep"), which allow them to learn hierarchical representations of data. Each layer extracts increasingly abstract features from the input data.\n\n3. **Learning fro

In [ ]:
qa.invoke({"query": "what is Automatic image captioning?"})

{'query': 'what is Automatic image captioning?',
 'result': 'Automatic image captioning is a deep learning application in computer vision that involves generating human-like textual descriptions for images. The process typically involves several steps:\n\n1. **Image Feature Extraction**: A deep convolutional neural network (CNN) is used to extract high-level features from the input image. These features capture essential information about objects, scenes, and textures in the image.\n\n2. **Sequence-to-Sequence Model**: The extracted image features are passed to a sequence-to-sequence model, often based on recurrent neural networks (RNNs) or transformer architectures. This model generates a sequence of words that form the image caption.\n\n3. **Language Model Training**: The RNN or transformer model is trained on a large dataset of paired images and captions. During training, the model learns the relationships between image features and corresponding textual descriptions.\n\n4. **Captio